This notebook ```beyond_baseline.ipynb``` is to fine-tune the model using AdamW optimizer to improve accuracy and loss, and eventually to overcome the baseline accuracy. For more information on data and structure, please refer to ```predictions.ipynb``` in this repository.


# Advanced Fine-Tuning: Optimizing Accuracy with AdamW
## Overview

This notebook serves as a dedicated implementation guide for tuning the AdamW (Adaptive Moments with Weight Decay) optimizer. It is designed as a direct follow-up to our initial model development phase.

The Goal: Move beyond "standard" training (SGD, Adam, adding ReLU) results by precisely calibrating the optimizer to improve validation accuracy, stabilize loss convergence, and prevent over-fitting during the fine-tuning process.

## Why AdamW for fine-tuning?

While standard Adam is effective, AdamW decouples weight decay from the gradient update. This is critical during fine-tuning because:

Weight Decay Recovery: It restores the original intent of $L_2$ regularization.

Generalization: It helps the model maintain the knowledge of pre-trained weights while adapting to new data.

Convergence: It often leads to lower final loss and higher accuracy compared to standard Adam when hyperparameters are properly tuned.


## Quick Summary: Why "Shallow-learning" with a single layer wins in genomics

The "Small N, Large P" Problem: You have ~30,000 features but only ~150 patients. A deep model has so many parameters that it will simply "memorize" your specific patients (like a lookup table) rather than learning actual biology. In other words, a deep model is prone to overfitting.

Additive Biology: In computer vision, pixels must form edges, then shapes, then objects. This needs many layers. In genomics, the relationship is often direct and additive: a specific set of genes being "on" or "off" directly signals a tumor. A single hidden layer captures this perfectly; adding more layers just adds noise.

The Curse of Dimensionality: In 30,000~dimensional space, data points are very spread out. It is mathematically easy to separate them with a single straight "wall" (a hyperplane). Shallow networks find these clean, straight boundaries. Deep networks try to "twist" the space, which usually results in a model that works on your data but fails on any new patient. Again, there is more possibility to overfit.

Signal vs. Noise: Biological data is very "noisy." Every extra layer in a neural network acts like a megaphone for that noise. A Shallow Network acts as a bottleneck, forcing the model to ignore the thousands of irrelevant genes and focus only on the strong biological signals.

In [1]:
# The learning tends to be better with fastai
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
# hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
import ssl, certifi
ssl._create_default_https_context = lambda *args, **kwargs: ssl.create_default_context(cafile=certifi.where())


# Recall: Data from patients with primary invasive breast cancer
This study examined tissue samples from patients with primary invasive breast cancer, including four major tumor subtypes: triple-negative (TN) (41 cases), HER2-positive (30 cases), luminal A (29 cases), and luminal B (30 cases). ***RNA, the molecule that reflects which genes are active in a cell, was extracted from all samples***. Gene activity across the entire genome was then measured using Affymetrix U133 Plus 2.0 microarray technology.

| Sample type   | Purpose                                            |
| ------------- | -------------------------------------------------- |
| Tumors        | Define subtype-specific cancer expression profiles |
| Normal tissue | Identify cancer-specific changes                   |
| Cell lines    | Enable experimental follow-up and model validation |


We obtained the data from National Institutes of Health (NIH), the National Center for Biotechnology Information, which is available at https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE45827.

In [4]:
import gzip
import pandas as pd
import urllib.request
from pathlib import Path

url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE45nnn/GSE45827/matrix/GSE45827_series_matrix.txt.gz"
path = Path("GSE45827_series_matrix.txt.gz")      # make sure that the file is saved in the same directory.

if not path.exists():
   urllib.request.urlretrieve(url, path)

# Data format from NCBI GEO
Data format: *NCBI GEO (Gene Expression Omnibus)*

File type: Series Matrix

We follow the NCBI GEO data format below: *"!series_matrix_table_begin"* (respectively, *"!series_matrix_table_end"*) is a standard marker defined by NCBI GEO, used in GEO Series Matrix files (*_series_matrix.txt) to mark the beginning of the expression data table.

We import the data directly from the path.

In [5]:
with gzip.open(path, "rt") as f:
    lines = f.readlines()

# Find where expression data starts
start = [i for i, l in enumerate(lines) if l.startswith("!series_matrix_table_begin")][0] + 1
end = [i for i, l in enumerate(lines) if l.startswith("!series_matrix_table_end")][0]

# "\t" for tab-separated data table
data = pd.read_csv(
    path,
    sep="\t",
    skiprows=start,
    nrows=end - start
)


# A suitable data frame
We use 'data.set_index(data.columns[0])' and return a new DataFrame (see the table below). After transpose, the first column of X is the sample ID (e.g., GSM1116084, GSM1116085, etc.) and the first row is the probe IDs (e.g., 1007_s_at, 1053_at, etc.). Probe IDs are identifiers for the physical DNA probes to detect gene expression. 

In [6]:
# First column is gene ID.
data = data.set_index(data.columns[0])

# Transpose: samples × genes
X = data.T

# number of (samples x genes) 
X.shape

(155, 29874)

The values of the table for given sample IDs and probe IDs represent gene-expression level — specifically, normalized microarray signal intensities measured by each probe for each sample. 

# Labels for training 

In [7]:
# Extract phenotype lines
pheno_lines = [l for l in lines if l.startswith("!Sample_source_name_ch1")]

labels = []
for line in pheno_lines:
    parts = line.strip().split("\t")[1:]
    labels = parts
    
# In total there are 155 labels.

In [8]:
yy = pd.Series(labels)

# Returns 1=true if the label contains 'tumor' and returns 0=false otherwise
y = pd.Series(labels).str.lower().str.contains("tumor").astype(int).values

As it shows above, there are 6 distinct labels: "Human Basal Tumor Sample", "Human CellLine", "Human Her2 Tumor Sample", "Human Luminal A Tumor Sample", "Human Luminal B Tumor Sample", "Human Normal". Now we assign boolean value to the labels which contain the word "tumor" to train the machine to study to give the right conclusions matching with the label. 

# Steps from ```predictions.ipynb```.
## Basic set-up for learning using two different methods: Stochastic gradient descent(SGD) and Adam optimizer.

Now we convert to torch.

In [9]:
import torch

# X is a pandas DataFrame here
X_df = X.copy()

X_df = (X_df - X_df.mean(axis=0)) / X_df.std(axis=0)
X_df = X_df.fillna(0.0)

# now convert to torch
X = torch.tensor(X_df.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)



In [10]:
n = len(X)
# The function randperm(n) generates a random ordering of integers from 1 to n
idx = torch.randperm(n)

train_idx = idx[:int(0.8*n)]
valid_idx = idx[int(0.8*n):]

train_x, train_y = X[train_idx], y[train_idx]
valid_x, valid_y = X[valid_idx], y[valid_idx]


# Principal Component Analysis

In [11]:
# We use sklearn for PCA and modify the PyTorch tensor to the right format to use sklearn

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt


# What is n_components in PCA? How can we choose optimal n_components?

pca = PCA(n_components=90)


In [12]:
X_np = train_x.detach().cpu().numpy()
X_np = StandardScaler().fit_transform(X_np)
X_pca = pca.fit_transform(X_np)
X_pca

array([[ 3.0765045e+01, -2.4614185e+01, -2.7316547e+01, ...,  3.6811919e+00, -1.8935709e+01, -1.5399693e+01],
       [-4.2143887e+01, -2.5798975e+01,  8.5787735e+01, ...,  6.0338507e+00,  1.4269685e+00,  1.3738943e+01],
       [-1.5060004e+02,  8.3600174e+01, -8.5271500e+01, ...,  9.2324817e-01, -1.1091703e+00,  2.0955551e+00],
       ...,
       [ 3.6703491e+01, -2.6457605e+01,  1.9464664e+01, ..., -1.3099131e+01, -6.3211486e-02,  1.3569843e+00],
       [-1.8587533e+00, -2.6162285e+01,  2.5469841e+01, ..., -1.3598913e+00, -1.2668482e+00, -3.8842266e+00],
       [ 9.9321854e+01, -1.1406039e+01, -1.6813551e+01, ...,  5.7678995e+00, -1.2250130e+01, -4.2018590e+00]], shape=(124, 90), dtype=float32)

We can see that 90 PCA components can explain 93% of the data.

In [13]:
print(np.round(pca.explained_variance_ratio_.cumsum(), 4))

[0.1416 0.2287 0.2929 0.3336 0.3639 0.3925 0.4147 0.4333 0.4496 0.4644 0.4781 0.4908 0.5034 0.5146 0.5256 0.5363 0.5465 0.5565 0.5664 0.5762 0.5854 0.5941 0.6026 0.6107 0.6187 0.6264 0.634  0.6416
 0.6491 0.6563 0.6635 0.6706 0.6776 0.6844 0.691  0.6975 0.7038 0.7101 0.7163 0.7224 0.7283 0.7341 0.7398 0.7455 0.7509 0.7564 0.7618 0.767  0.7722 0.7773 0.7823 0.7872 0.792  0.7967 0.8014 0.806
 0.8105 0.815  0.8194 0.8238 0.828  0.8322 0.8364 0.8405 0.8445 0.8484 0.8524 0.8562 0.8599 0.8636 0.8673 0.8709 0.8745 0.878  0.8815 0.885  0.8884 0.8917 0.895  0.8982 0.9014 0.9046 0.9077 0.9108
 0.9138 0.9168 0.9198 0.9227 0.9255 0.9283]


In [14]:
pca.fit(X_pca)

PCA(n_components=90)

# Train the model using SGD and Adam optimizer

In [22]:
from torch.utils.data import DataLoader
from fastai.data.core import DataLoaders


bs = 8

train_dl = DataLoader(list(zip(train_x, train_y)), batch_size=bs, shuffle=True)
valid_dl = DataLoader(list(zip(valid_x, valid_y)), batch_size=bs)

# dls = DataLoaders(train_dl, valid_dl)




In [23]:
import torch.nn as nn
from torch.optim import SGD, Adam

Tumor samples have label y = 1
Normal samples have label y = 0. 
Thus we use the linear model
$$z=w^⊤ x+b,$$
where the weight $w=(w_1,w_2, \ldots, w_m)$ has the number of entries which corresponds to the number of probes $m$ and $b$ represents bias. The way that we defined the label, the signs and the absolute value of the weights suggest the following:
* $w_i>0$: higher expression pushes toward tumor;
* $w_i<0$: higher expression pushes toward normal;
* Large $|w_i|$: this probe contributes strongly to the decision direction;
* Small $|w_i|$: this probe carries no predictive signal.

As we are building a model which can perform a binary classification, i.e., whether a given sample has tumor (value 1) or normal (value 0), we use the nn.BCEWithLogitLoss() loss function. 
Binary cross-entropy is defined per sample:

$$\ell(x, y)
= -\big[ y \log p(x) + (1 - y)\log(1 - p(x)) \big],$$
where $p(x)=P(y=1|x)=\sigma(w^⊤ x+b)$, and $\sigma$ is the sigmoid function.
And the model objective is the average loss over 
$N$ elements ($N$ is the batch size.):
$$L
= \frac{1}{N} \sum_{i=1}^{N} \ell(x_i, y_i)$$


## Adding nonlinearity to the model
We quickly recall nn.Sequential, a container module in PyTorch designed to stack neural network layers in a specific, ordered sequence. 
```python \n
model = nn.Sequential(
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 1)
)
```
Then calling model(x) is equivalent to

```python \n
x = nn.Linear(100,50)(x)
x = nn.ReLU()(x)
x = nn.Linear(50,1)(x)
```

## Layers and the change of the shape of the input

We add Fully-connected layer to improve our model. In particular, we updated the linear model using ```.ReLU()```. We add layers as follows: 
```python \n
model = nn.Sequential(
        nn.Linear(X.shape[1], 64),
        nn.ReLU(),
        nn.Linear(64, 1)
        )
```
We defined previously train_dl, the dataloader with batch size (bs=32) as follows.
```python \n
train_dl = DataLoader(list(zip(train_x, train_y)), batch_size=bs, shuffle=True)
```
Note that we have 124 pairs in ```train_dl``` with batches of size 32, 32, 32, 28.

We recall that our baseline is ```y.mean()=0.8387```. Thus the model from ```predictions.ipynb``` cannot improve its accuracy beyond the baseline. Now we make the model deeper to make improvements.

In [24]:
loss_func = nn.BCEWithLogitsLoss()
y.mean()

tensor(0.8387)

To improve the model, we try adding more layers to the model. On the other hand, as it shows below, overfitting happens.

In [25]:
def batch_accuracy(preds, y, thresh=0.5):
    return ((preds.sigmoid() > thresh) == y).float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [26]:
def train_model_more_layer(opt, epochs=20):
    
    model = nn.Sequential(
        nn.Linear(X.shape[1],128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Linear(16,8),
        nn.ReLU(),
        nn.Linear(8,4),
        nn.ReLU(),
        nn.Linear(4,1)
    )  

    
#   We had previously only the following single layer     
#   model = nn.Linear(X.shape[1], 1) 
#   With the single-layer model, model(xb) is of torch size model(xb).shape=[28,1] and yb is of torch size [28]


    # The folloing line nstantiates the optimizer, passing it the model’s trainable parameters
    # opt = SGD(model.parameters()) is equivalent to train_model(SGD)
    opt = opt(model.parameters())

    for epoch in range(epochs):
        total_loss = 0
        for xb, yb in train_dl:
            
            # Recall that loss_func = nn.BCEWithLogitsLoss()
            
            loss = loss_func(model(xb).squeeze(), yb)
            loss.backward()
            
            opt.step()
            opt.zero_grad()
            
            total_loss += loss.item()
            probs = model(valid_x).sigmoid()
        
        print(f"Epoch {epoch}: loss={total_loss:.3f}, acc={validate_epoch(model)}", model(xb).shape)
#        print(probs.min().item(), probs.max().item(), probs.mean().item())
    
    return model    

In [27]:
print("more layer")
model_more_layer = train_model_more_layer(
    lambda params: Adam(params, lr=1e-3, weight_decay=1e-3)
)

more layer
Epoch 0: loss=5.817, acc=0.7825 torch.Size([4, 1])
Epoch 1: loss=1.575, acc=0.7357 torch.Size([4, 1])
Epoch 2: loss=0.416, acc=0.7357 torch.Size([4, 1])
Epoch 3: loss=0.042, acc=0.7357 torch.Size([4, 1])
Epoch 4: loss=0.030, acc=0.7357 torch.Size([4, 1])
Epoch 5: loss=0.047, acc=0.7357 torch.Size([4, 1])
Epoch 6: loss=0.043, acc=0.7357 torch.Size([4, 1])
Epoch 7: loss=0.027, acc=0.7357 torch.Size([4, 1])
Epoch 8: loss=0.024, acc=0.7357 torch.Size([4, 1])
Epoch 9: loss=0.017, acc=0.7357 torch.Size([4, 1])
Epoch 10: loss=0.017, acc=0.7357 torch.Size([4, 1])
Epoch 11: loss=0.011, acc=0.7357 torch.Size([4, 1])
Epoch 12: loss=0.012, acc=0.7357 torch.Size([4, 1])
Epoch 13: loss=0.010, acc=0.7357 torch.Size([4, 1])
Epoch 14: loss=0.010, acc=0.7357 torch.Size([4, 1])
Epoch 15: loss=0.009, acc=0.7357 torch.Size([4, 1])
Epoch 16: loss=0.008, acc=0.7357 torch.Size([4, 1])
Epoch 17: loss=0.008, acc=0.7357 torch.Size([4, 1])
Epoch 18: loss=0.006, acc=0.7357 torch.Size([4, 1])
Epoch 19: l

# Learning with data filtered by the PCA 
We use SGD+ReLU (same as above) on the data filtered by the PCA.

In [15]:
train_x_pca = pca.fit_transform(train_x)   # (124, 85)
valid_x_pca = pca.transform(valid_x)

In [16]:
train_x_pca = torch.tensor(train_x_pca, dtype=torch.float32)
valid_x_pca = torch.tensor(valid_x_pca, dtype=torch.float32)

In [17]:
X_pca = torch.tensor(X_pca)
# X_pca = X_pca.detach().clone()
X_pca.shape

torch.Size([124, 90])

In [18]:
bs=8

train_dl = DataLoader(
    list(zip(train_x_pca, train_y)),
    batch_size=bs,
    shuffle=True
)

valid_dl = DataLoader(
    list(zip(valid_x_pca, valid_y)),
    batch_size=bs
)

In [19]:
train_x_pca.shape, valid_x_pca.shape, train_y.shape, valid_y.shape

(torch.Size([124, 90]),
 torch.Size([31, 90]),
 torch.Size([124]),
 torch.Size([31]))

In [20]:
loss_func = nn.BCEWithLogitsLoss()

def batch_accuracy_sm(preds, y, thresh=0.5):
    # .squeeze() ensures [batch_size, 1] becomes [batch_size]
    return ((preds.sigmoid().squeeze() > thresh) == y).float().mean()

def validate_epoch_sm(model):
    model.eval() # Always set to eval mode for validation
    with torch.no_grad(): # Disable gradient calculation
        accs = [batch_accuracy_sm(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

We make few changes to prevent overconfidence and make the model more stable. First, we smoothen the label. Instead of targets being strictly binary, between 0 and 1, we move them slightly toward the middle (0.5). For a smoothing value of ```eps=0.1```, 0 becomes 0.05 and 1 becomes 0.95. We also added ```nn.Dropout(0.3)``` to prevent overfitting.

In [21]:
def train_model_smoothed(opt, epochs=20, eps=0.1):
    
    # A shallower model is less likely to "memorize" noise in small genomic datasets

    model = nn.Sequential(
        nn.Linear(X_pca.shape[1], 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 1)
    )  
    
    opt = opt(model.parameters())
    # Notice: No label_smoothing argument here anymore
    loss_func = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in train_dl:
            # --- Manual Label Smoothing Step ---
            # Formula: smoothed = y * (1 - eps) + 0.5 * eps
            yb_smoothed = yb * (1 - eps) + (0.5 * eps)
            
            logits = model(xb).squeeze()
            loss = loss_func(logits, yb_smoothed)
            
            loss.backward()
            opt.step()
            opt.zero_grad()
            total_loss += loss.item()
            
        acc = validate_epoch_sm(model)
        if epoch % 1 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch}: Loss={total_loss/len(train_dl):.4f}, Acc={acc:.4f}")
    
    return model

# Running with manual smoothing
model_smooth = train_model_smoothed(
    lambda params: torch.optim.AdamW(params, lr=1e-3, weight_decay=0.1),
    eps=0.1  # 10% smoothing
)

Epoch 0: Loss=1.9600, Acc=0.9375
Epoch 1: Loss=0.5031, Acc=1.0000
Epoch 2: Loss=0.4371, Acc=1.0000
Epoch 3: Loss=0.3993, Acc=0.9688
Epoch 4: Loss=0.3865, Acc=1.0000
Epoch 5: Loss=0.3549, Acc=1.0000
Epoch 6: Loss=0.3817, Acc=1.0000
Epoch 7: Loss=0.4335, Acc=1.0000
Epoch 8: Loss=0.3029, Acc=0.9688
Epoch 9: Loss=0.3434, Acc=0.9688
Epoch 10: Loss=0.3297, Acc=0.9688
Epoch 11: Loss=0.3481, Acc=1.0000
Epoch 12: Loss=0.3178, Acc=1.0000
Epoch 13: Loss=0.3101, Acc=1.0000
Epoch 14: Loss=0.3055, Acc=1.0000
Epoch 15: Loss=0.3057, Acc=1.0000
Epoch 16: Loss=0.2920, Acc=1.0000
Epoch 17: Loss=0.2777, Acc=1.0000
Epoch 18: Loss=0.2930, Acc=1.0000
Epoch 19: Loss=0.2922, Acc=1.0000
